<a href="https://colab.research.google.com/github/nlpbokproject/NLP_BOK_Team/blob/master/Tone_%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import konlpy as kn
from ekonlpy.tag import Mecab
import ekonlpy as ek
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from glob import glob

In [0]:
df_dic = pd.read_csv('../DATA/NBC_step3_final-jh.csv',encoding='UTF-8', names=['hawki','dovis'])
#딕셔너리 파일을 불러옴
hawki_ls = df_dic['hawki']
#매파 단어들에 대한 리스트 생성
dovis_ls = df_dic['dovis']
#비둘기파 단어들에 대한 리스트 생성

In [0]:
df = pd.read_csv('../DATA/minutes_preprocessed.csv', encoding='utf-8', names=['date','minutes','Economic Situation','Foreign Currency','Financial Markets','Monetary Policy','Participant Views','Government View','Economic Situation count','Foreign Currency count','Financial Markets count','Monetary Policy count','Participant Views count','Government View count','Foreign Currency tokens','Financial Markets tokens','Foreign Currency ngram','Financial Markets ngram'])
#정제된 의사록 데이터 불러옴
df = df.sort_values(by = ['date'], axis=0)
#날짜 순으로 정렬
df = df.drop(['Economic Situation','Monetary Policy','Participant Views','Government View', 'Economic Situation count','Monetary Policy count','Participant Views count','Government View count'], axis=1)
df = df.drop(['minutes','Foreign Currency count','Financial Markets count','Foreign Currency tokens', 'Financial Markets tokens','Foreign Currency ngram','Financial Markets ngram'],axis=1)
#의사록 데이터의 섹션 2,3만 사용
df = df.drop(0)
df

,date,Foreign Currency,Financial Markets
1,20050609,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...
2,20050707,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...
3,20050811,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...
4,20050908,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...
5,20051011,일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...
6,20051110,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",일부 위원은 장단기금리 격차가 경제성장률에 선행하는 모습을 보인다고 하면 미국의...
7,20051208,일부 위원은 금년 들어 달러화가 엔화 나 유로화에 대해 상당히 강세를 나타낸 것과...,일부 위원은 우리나라와 미국의 장기시장금리 를 비교해볼 때 최근 우리나라의 장기시...
8,20060112,일부 위원은 2002년부터 거의 예외 없 이 연초에는 원화의 강세 현상이 나타나 ...,일부 위원은 실물활동 중 민간소비나 GDP 증가율이 M1증가율과 밀접한 관련성이...
9,20060209,일부 위원은 자본유입에 대해서는 관대 한 반면 자본유출에 대해서는 비교적 엄격한 ...,일부 위원은 최근 주가가 크게 변동한 이유 중 하나로 기관투자가들이 단기적 관점...
10,20060309,일부 위원은 과거 사례를 보면 원화환 율은 엔화환율 및 위안화환율과 높은 상관관계...,일부 위원은 지난 해 10월 이후 세 차례에 걸 친 콜금리 목표 인상에도 불구하고...


In [0]:
For_ls = df['Foreign Currency']
Fin_ls = df['Financial Markets']
tmp_ls = For_ls + Fin_ls
df['For+Fin'] = tmp_ls
df = df.dropna()
df = df.drop(columns=['Foreign Currency', 'Financial Markets'])
tmp_ls[1]

'일부 위원은 우리나라 경제가 일본경제  에 비해 더 나아질 특별한 요인이 없음에도 불구하고 최근 원/엔 환율이 지속적으  로 하락하는 이유를 물었으며, 이에 대해 관련부서에서는 위안화 절상시 동아시아  권 통화의 절상폭이 클 것이라는 기대로 원화나 대만달러가 엔/달러 환율의 상승  을 추수하지 못하고 국내 외환수급 면에서 공급우위 상태가 지속되고 있는 데다   국제 투자은행들이 엔/달러 환율 전망을 상향조정하고 있음에도 불구하고 원/달러   환율에 대한 조정은 아직 이루어지지 않고 있어 기업 등 시장참가자들의 기대도   원화 절상 쪽으로 편향되어 있는 데 기인한다고 설명하였음.@@@- 6 -     동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamental)을 반영하  여 움직인다는 점을 감안할 때 현재의 원/엔 환율 하락은 일시적인 현상이며 장기  적으로 반전될 가능성이 있는 것으로 보인다는 견해를 밝혔으며, 이에 대해 관련  부서에서는 동 견해에 동의하면서 우리나라의 경우 시장규모가 작아 일시적인 외  부 충격에 의한 영향을 쉽게 받는다는 점을 설명하는 한편, 향후 우리나라와 미국  간 내외금리 역전이 나타나고 작년과 같은 대규모 외환 유입은 일어나지 않을 것  임을 감안할 때 원/엔 환율이 추가 하락할 여지는 점차 축소될 것으로 생각된다고   답변하였음.@@@다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운  데 일각에서는 정책금리 인하론까지 대두되면서 유로화가 크게 절하되고 있다면  서 유로화의 달러화에 대한 흐름이 중장기적으로 변화하는 것으로 볼 수 있는지   물었으며, 이에 대해 관련부서에서는 국제 투자은행들이 달러화 대비 유로화 전망  을 기존의 강세에서 약세로 바꾸고 있으며 장기적으로 다시 강세로 반전되지는   않을 것이라는 의견이 지배적이라고 설명하였음.@@@동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운  데 달러화 보유에 따른 환차익 기대가 형성될 경우 원화 강세에 대한 시장의 기 

In [0]:
a = script_ls[10].split('.')
for i in a:
    print(i +'\n\n')

일부 위원은 과거 사례를 보면 원화환  율은 엔화환율 및 위안화환율과 높은 상관관계를 갖고 있는데 국제금융시장에서  는 엔화와 위안화가 금년중 절상될 것으로 예상하고 있다면서 엔화 및 위안화가   절상되는 경우 원화환율에 미치는 영향에 대해 물었음


@@@이에 대해 관련부서에서는 2004년 이후 원화환율과 엔화환율간의 상관관계  가 상당히 약해졌으며 조만간 일본이 금리를 인상하게 되면 엔 캐리 트레이드(Yen   carry-trade) 현상도 사라지고 그에 따라 우리나라에 유입된 엔 캐리 트레이드 관  련자금의 유출이 예상되어 엔화 강세가 원화 강세로 연결되지 않을 수 있으며, 중  국 위안화가 절상되더라도 지난 해 7월 위안화 절상시 하락하였던 원화환율이 수  일내 회복된 과거 경험에 비추어 볼 때 단기적으로 원화가치에 영향을 주겠지만   평균 환율의 하락압력으로 작용할 여지는 크지 않을 것으로 생각된다고 답변하였  음


@@@동 위원은 최근 중국 국채의 가산금리가 우리나라 외평채의 가산금리보다   낮게 나타나는 이유에 대해 물었으며, 이에 대해 관련부서에는 중국의 금융부문이   취약한 것으로 알려졌음에도 불구하고 외환보유고가 세계 제1위인 점에도 일부   기인하지만 기본적으로 중국 외화표시 국채는 유통물량도 적을 뿐만 아니라 중국  계가 대부분 매입하고 있어 중국 국채의 가산금리가 엄밀한 의미에서 국가 신용  도를 반영한 것으로 보기 어려운 것으로 생각된다고 답변하였음


@@@다른 일부 위원은 개발도상국에 유입되었던 자금이 미국, 유럽, 일본 등 선  진국의 금리인상으로 선진국으로 유출되는 경우 우리나라의 외환 및 증권 시장에   좋지 않은 영향을 줄 수 있다는 일부 우려에 대한 관련부서의 견해를 물었음


@@@이에 대해 관련부서에서는 저금리에서 고금리로의 이행에 따른 파급효과   및 충격은 단기 투기성 자금 규모에 크게 좌우된다는 점에서 엔 캐리 트레이드     - 4 -  자금의 이동에 대해 주목할 필요가 있다면서, 최근 엔 캐리 트레이드 자금의

In [0]:
from ekonlpy.sentiment import MPKO
def determiner_s(sentence, hawki_ls, dovis_ls):
    #문장의 톤을 구하는 함수. (문장 하나가 이 함수의 첫번째 변수로 들어옴.)
    hawki_cnt = 0
    dovis_cnt = 0
    mpko = MPKO(kind=1)
    tokens = mpko.tokenize(sentence)
    #문장별로 토큰화
    #print(tokens)
    for token in tokens:
        if token in hawki_ls.values:
            hawki_cnt += 1
            print(token+" hawk")
            #토큰이 매파에 해당하면 매파 개수 카운트 +1
        if token in dovis_ls.values:
            dovis_cnt += 1
            print(token+" dovis")
            #토큰이 비둘기파에 해당하면 비둘기파 개수 카운트 +1
    if hawki_cnt > dovis_cnt:
        return 1
        #매파 토큰이 더 많은 경우.
    elif hawki_cnt < dovis_cnt:
        return 0
        #비둘기파 토큰이 더 많은 경우.
    else:
        return -1
        #매파, 비둘키파 토큰의 수가 각각 같은 경우.

def determiner_p(p, hawki_ls, dovis_ls):
    tmp = p.split('.')
    # 마침표(.)을 단위로 문장으로 나눔
    hawki_cnt = 0
    dovis_cnt = 0
    for sentence in tmp:
        print(sentence)
        det = determiner_s(sentence, hawki_ls, dovis_ls)
        #나눠진 문장을 문장톤을 구하는 함수(본 함수의 위 함수)에 변수로 넣어 실행
        if det == 1:
            hawki_cnt +=1
            #매파 문장으로 판별 경우, hawk_cnt +1
        elif det == 0:
            dovis_cnt += 1
            #비둘기파 문장으로 판별 경우, dovis_cnt +1
        else:
            continue
    print('hawki : ' + str(hawki_cnt))
    print('dovis : ' + str(dovis_cnt))
    print("\nsentence end")
    try:
        tone = (hawki_cnt - dovis_cnt)/(hawki_cnt + dovis_cnt)
        #문서 톤 구하기
        return tone
    except:
        print("tone error")
        return None

In [0]:
a = determiner_p('한국감정원 강여정 부장은 "정부의 11·3 대책 발표와 미국 기준금리 인상, 대출 규제 이후 주택시장이 관망세로 접어든 상태"라며 "계절적 비수기에 다 국내 산업경기 회복 지연, 공급과잉 우려 등까지 겹치면서 본격적인 봄이사철이 오기 전까지는 주택 가격이 상승하기 어려울 것"이라고 말했다', hawki_ls, dovis_ls)
print(a)
#임의 데이터로 테스트

In [0]:
script_ls = df['For+Fin']
tone_ls = []
for p in script_ls:
    p = p.replace('@@@','')
    tone = determiner_p(p,hawki_ls, dovis_ls)
    #determiner_p 함수로 문서 전체에 대한 톤 구함
    tone_ls.append(tone)
    #임시 리스트에 tone 저장
    print(tone)

일부 위원은 우리나라 경제가 일본경제  에 비해 더 나아질 특별한 요인이 없음에도 불구하고 최근 원/엔 환율이 지속적으  로 하락하는 이유를 물었으며, 이에 대해 관련부서에서는 위안화 절상시 동아시아  권 통화의 절상폭이 클 것이라는 기대로 원화나 대만달러가 엔/달러 환율의 상승  을 추수하지 못하고 국내 외환수급 면에서 공급우위 상태가 지속되고 있는 데다   국제 투자은행들이 엔/달러 환율 전망을 상향조정하고 있음에도 불구하고 원/달러   환율에 대한 조정은 아직 이루어지지 않고 있어 기업 등 시장참가자들의 기대도   원화 절상 쪽으로 편향되어 있는 데 기인한다고 설명하였음
원엔/NNG;환율/NNG;하락/NNG dovis
기대/NNG;원화/NNG;절상/NNG dovis
통화/NNG;절상/NNG dovis
- 6 -     동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamental)을 반영하  여 움직인다는 점을 감안할 때 현재의 원/엔 환율 하락은 일시적인 현상이며 장기  적으로 반전될 가능성이 있는 것으로 보인다는 견해를 밝혔으며, 이에 대해 관련  부서에서는 동 견해에 동의하면서 우리나라의 경우 시장규모가 작아 일시적인 외  부 충격에 의한 영향을 쉽게 받는다는 점을 설명하는 한편, 향후 우리나라와 미국  간 내외금리 역전이 나타나고 작년과 같은 대규모 외환 유입은 일어나지 않을 것  임을 감안할 때 원/엔 환율이 추가 하락할 여지는 점차 축소될 것으로 생각된다고   답변하였음
원엔/NNG;환율/NNG;하락/NNG dovis
하락/NNG;일시적/VAX dovis
다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운  데 일각에서는 정책금리 인하론까지 대두되면서 유로화가 크게 절하되고 있다면  서 유로화의 달러화에 대한 흐름이 중장기적으로 변화하는 것으로 볼 수 있는지   물었으며, 이에 대해 관련부서에서는 국제 투자은행들이 달러화 대비 유로화 전망  을 기존의 강세에서 약세로 바꾸고 있으며 장기적으로 다시 강세로 반전되지는 

투자/NNG;감소/NNG hawk
플러스/NNG hawk
셋째, 작년 금리인하의   파급효과로 대출이 늘어나고 있는 것으로 보이나 소비, 투자 등에 미치는 영향에   대한 파악이 어려운 상태에서 부동산시장 불안 등 국민경제에 부정적인 측면이   나타나면서 정부가 대응조치를 취함에 따라 금리인하의 효과가 일부 차단되는 결  과가 빚어지고 있는 것으로 보임
인하/NNG;파급효과/NNG dovis
인하/NNG;효과/NNG dovis
넷째, 일부지역에서 나타난 주택가격 상승이 주  변지역으로 확산되고 있는 반면, 집세는 안정되어 있어 주택가격의 상당부분이   거품일 가능성이 높음
거품/NNG;가능성/NNG;높/VA dovis
주택가격/NNG;거품/NNG hawk
한편, 아파트나 토지에서 발생하고 있는 막대한 개발이익  이 대부분 민간부문에 남아 투기자금화하는 문제를 해결하기 위해 개발이익의 상  당부분을 공공부문에 환수시켜 공공부문이 주택 공급을 추가로 확대시켜 나가는   선순환구조로 흐름을 바꾸는 정책이 필요함
주택/NNG;공급/NNG;확대/NNG hawk
다섯째, 중도금대출 대상을 1가구 1  주택자 또는 무주택자로 한정하고 집단대출을 한시적으로 억제 또는 금지하는 등  의 제한조치가 현행 법제상 가능한지 여부를 다각적으로 검토할 필요가 있음
여  섯째, 정책조합(policy-mix) 면에서 추경 논의가 있고 그 당위성에 대해서는 인정  하나 재정의 효율성이 낮기 때문에 추경의 효과는 그만큼 제약될 것으로 보임
이와 같은 위원들간의 견해 표명에 따라 위원들은 콜금리 목표를 현 수준    - 12 -  에서 유지하는 방향으로 구체적인 의결문안을 작성하였으며, 이에 대해 위원들이   모두 동의하였음
(４) 심의결과      의결문 작성․가결          의결사항  ｢통화정책방향｣을 다음과 같이 결정한다
□ 실물경제는 수출이 증가세를 유지하는 가운데 민간소비의 회복 움직임
이 이어지고 건설투자의 부진도 다소 완화되는 기미를 보이고 있으나   설비투자의 개선은 뚜렷하

KeyboardInterrupt: 

In [0]:
tmp_ls #문서별 tone들이 저장된 임시 리스트

In [0]:
tone_ls #문서별 tone들이 저장된 임시 리스트

In [0]:
df['tone'] = tone_ls #dataframe에 tone column 추가.

In [0]:
df

In [0]:
df.to_csv("../DATA/tone-test-hs.csv")

In [0]:
df2 = df[['date', 'tone']]
df2.to_csv("../DATA/tone-test-simple-190529.csv") #date와 tone만 저장된 파일입니다
df2

In [0]:
df